<a href="https://colab.research.google.com/github/karyam/rgnn_eeg_emotion_classifier/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dgl
!pip install -U mne

In [ ]:
import tensorflow as tf
print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

In [3]:
!pip install tfdlpack

     |████████████████████████████████| 71kB 2.0MB/s 


In [1]:
# sometimes this cell needs to be run multiple times idk why
!python -m dgl.backend.set_default_backend tensorflow
#!python -m dgl.backend.set_default_backend pytorch

Using backend: tensorflow
2020-10-13 11:51:10.553426: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-13 11:51:12.480573: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-13 11:51:12.545970: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-13 11:51:12.546045: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (57bf1ddcfed0): /proc/driver/nvidia/version does not exist
2020-10-13 11:51:12.591276: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2020-10-13 11:51:12.591571: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x27aea00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-13 11:51:1

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import pandas as pd
import os, sys
import tensorflow as tf
from tensorflow import keras
import tensorboard
import dgl
import scipy.io as io
from sklearn.model_selection import train_test_split
from dgl.data.utils import *
from dgl.nn import GraphConv
import tensorflow_data_validation as tfdv
import seaborn as sns
import matplotlib.pyplot as plt

In [49]:
#globals
FEATURES_DATA_PATH = "drive/My Drive/BCI/code/data/SEED/ExtractedFeatures"
PREPROCESSED_DATA_PATH = "drive/My Drive/BCI/code/data/SEED/Preprocessed_EEG"
CSV_PATH = "drive/My Drive/BCI_clone/CSV"
SUBJECT_NPY_DATA_PATH = "drive/My Drive/BCI_clone/npy/npy_0.npy"
SUBJECT_NPY_LABEL_PATH = "drive/My Drive/BCI_clone/npy/npy_0_label.npy"
SUBJECT_CSV_PATH = CSV_PATH + "/" + "csv_0.csv"
ALL_SUBJECTS_CSV_PATH = CSV_PATH + "/" + "csv_all_subjects.csv"
ALL_SUBJECTS_NPY_DATA_PATH = "drive/My Drive/BCI_clone/npy/npy_all_subjects.npy"
ALL_SUBJECTS_NPY_LABEL_PATH = "drive/My Drive/BCI_clone/npy/npy_all_subjects_label.npy"
GRAPH_PATH = "drive/My Drive/BCI_clone/graph/"
CHECKPOINTS_PATH = "drive/My Drive/BCI_clone/checkpoints/"

# sampling frequency
sf = 200

# hyper-parameters
num_trials = 15
num_subjects = 15
num_bands = 5
num_classes = 3

In [ ]:
# get stats per subject using tfdv
subject_stats = tfdv.generate_statistics_from_csv(data_location=SUBJECT_CSV_PATH)

In [ ]:
tfdv.visualize_statistics(subject_stats)

#### Analyse feature correlations

In [ ]:
subject_df = pd.read_csv(SUBJECT_CSV_PATH)

In [ ]:
# get correlated features
corr_features = set()
corr_matrix = subject_df.corr()
# visualization doesn't help since there are way to many features
# plt.figure(figsize=(100,100))
# sns.heatmap(corr_matrix)

In [ ]:
# get features most correlated with the label
# bands: gamma and beta show meaningful corelation with EEG-based emotion classification
# as specified by https://www.researchgate.net/publication/276443876_Investigating_Critical_Frequency_Bands_and_Channels_for_EEG-Based_Emotion_Recognition_with_Deep_Neural_Networks
pos_corr_features = corr_matrix.sort_values(by=["label"], ascending=False)["label"].head()
neg_corr_features = corr_matrix.sort_values(by=["label"])["label"].head()

print(pos_corr_features)
print()
print(neg_corr_features)


In [ ]:
for i in range(len(subject_df.columns)):
  for j in range(i):
    if abs(corr_matrix.iloc[i,j]) > 0.8 and subject_df.columns[i] != "label" and subject_df.columns[j] not in pos_corr_features + neg_corr_features: 
      corr_features.add(corr_matrix.columns[i])

print(len(corr_features))
subject_df.drop(labels=corr_features, axis=1, inplace=True)

In [ ]:
len(subject_df.columns)

In [ ]:
def get_de_lds_from_csv(df):
  X = df.sample(frac=1).reset_index(drop=True) # shuffle dataset
  Y = X["label"].to_numpy()
  X.drop(labels="label", axis=1, inplace=True)
  X.drop(labels=df.columns[0], axis=1, inplace=True)  # drop the index column
  X.drop(labels=df.columns[62*5+1:-1], axis=1, inplace=True) # take only de_lds features
  X = X.to_numpy()
  return X, Y

### Baseline classification (MLP)

#### Subject-dependent training


In [24]:
model = tf.keras.Sequential([
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(num_classes)
])

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
X = np.load(SUBJECT_NPY_DATA_PATH)
Y = np.load(SUBJECT_NPY_LABEL_PATH)
X = np.reshape(X, (-1, 62*25))
print(X.shape, Y.shape)
# do not forget to shuffle samples
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2025, 1550) (2025,)
(1620, 1550) (1620,)
(405, 1550) (405,)


In [28]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4827 - accuracy: 0.7858
Epoch 2/200
51/51 [==============================] - 0s 2ms/step - loss: 0.5433 - accuracy: 0.7438
Epoch 3/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4958 - accuracy: 0.7907
Epoch 4/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4874 - accuracy: 0.7809
Epoch 5/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.8043
Epoch 6/200
51/51 [==============================] - 0s 3ms/step - loss: 0.4631 - accuracy: 0.7969
Epoch 7/200
51/51 [==============================] - 0s 3ms/step - loss: 0.6260 - accuracy: 0.7284
Epoch 8/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4682 - accuracy: 0.7975
Epoch 9/200
51/51 [==============================] - 0s 3ms/step - loss: 0.4369 - accuracy: 0.8074
Epoch 10/200
51/51 [==============================] - 0s 2ms/step - loss: 0.4235 - accuracy: 0.8272
Epoch 11/

In [29]:
#evaluate subject-dependent mlp: accuracy ~90%
history = model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 1ms/step - loss: 0.2757 - accuracy: 0.8988


#### Subject-independent training

In [30]:
model = tf.keras.Sequential([
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(num_classes)
])

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [32]:
X = np.load(ALL_SUBJECTS_NPY_DATA_PATH)
Y = np.load(ALL_SUBJECTS_NPY_LABEL_PATH)
print(X.shape, Y.shape)
X = np.reshape(X, (-1, 62*25))
print(X.shape, Y.shape)
# do not forget to shuffle samples
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30375, 62, 25) (30375,)
(30375, 1550) (30375,)
(24300, 1550) (24300,)
(6075, 1550) (6075,)


In [33]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
760/760 [==============================] - 2s 3ms/step - loss: 1.1626 - accuracy: 0.3775
Epoch 2/200
760/760 [==============================] - 2s 3ms/step - loss: 1.0239 - accuracy: 0.4558
Epoch 3/200
760/760 [==============================] - 2s 3ms/step - loss: 0.9595 - accuracy: 0.4921
Epoch 4/200
760/760 [==============================] - 2s 3ms/step - loss: 0.9510 - accuracy: 0.5044
Epoch 5/200
760/760 [==============================] - 2s 3ms/step - loss: 0.9261 - accuracy: 0.5185
Epoch 6/200
760/760 [==============================] - 2s 3ms/step - loss: 0.9038 - accuracy: 0.5287
Epoch 7/200
760/760 [==============================] - 2s 3ms/step - loss: 0.8814 - accuracy: 0.5474
Epoch 8/200
760/760 [==============================] - 3s 3ms/step - loss: 0.8931 - accuracy: 0.5360
Epoch 9/200
760/760 [==============================] - 4s 5ms/step - loss: 0.8688 - accuracy: 0.5514
Epoch 10/200
760/760 [==============================] - 2s 2ms/step - loss: 0.8573 - accura

In [ ]:
#evaluate subject-independent mlp: accuracy ~35%
history = model.evaluate(X_test, y_test)

### Baseline Graph Convolution Network

* **Input** - DGL Graph object
* **Output** - predicted logits per each graph
* **Layers** - GC64M16 - GC64M16 - P2 - GC128M9 - GC128M9 - P2 - FC40

#### Build input graphs

In [63]:
#TODO: Test with multiple bi-variate metrics for edge features
def correlation(signal1, signal2):
  return abs(np.corrcoef(signal1, signal2)[0][1])

In [64]:
def signal_2_graph(signal, treshold=0.6, channel_dist=correlation, num_channels=62):
  """
  Function to create one graph data sample (window)
  Arguments:
    - signal (ndarray(num_ch, num_bands*num_features)): signal(features) from a single window
    - treshold
    - channel_dist: distance measure between 2 channels
    - num_channels
  Return:
    - graph (DGLGraph object) 
  """

  src = []
  dest = []
  edgef = []

  for i in range(num_channels-1):
    for j in range(i+1,num_channels):
      weight = channel_dist(signal[i], signal[j])
      if weight > treshold:
        src.append(i)
        dest.append(j)
        edgef.append(weight)
        #dgl edges are in stored of their addition

  #make edges bi-directional for undirected graph
  u = np.concatenate([src, dest])
  v = np.concatenate([dest, src])
  graph = dgl.DGLGraph((u, v))
  
  assert graph.number_of_nodes() == num_channels 
  graph.add_edges(graph.nodes(), graph.nodes()) # add self-loops
  
  signal = tf.convert_to_tensor(signal, dtype='float32')
  graph.ndata['x'] = signal
  edgef = np.concatenate([edgef, edgef, [1]*num_channels])
  
  assert edgef.shape[0] == graph.number_of_edges()
  
  edgef = tf.convert_to_tensor(edgef, dtype='float32')
  graph.edata['w'] = edgef
  return graph

In [65]:
#TODO: Fix the labels dim mismatch problem
def generate_subject_graphs(data, labels, subject):
  """
  Arguments:
    - data: type(ndarray), shape(num_samples, num_channels, num_bands)
  Return:
    - graphs: type(ndarray of DGLGraph objects) shape(num_samples,)
  """
  graphs = []
  graph_labels = {"glabels": tf.convert_to_tensor(labels)}
  num_samples = data.shape[0]
  
  for i in range(num_samples):
    graphs.append(signal_2_graph(data[i]))
  save_graphs(GRAPH_PATH+f'graphs_from_de_feature_subj_{subject}.bin', graphs, graph_labels)
  return graphs

In [ ]:
X = np.load(SUBJECT_NPY_DATA_PATH)
Y = np.load(SUBJECT_NPY_LABEL_PATH)
graph_data = generate_subject_graphs(X, Y, subject=0)

In [ ]:
X = np.load(ALL_SUBJECTS_NPY_DATA_PATH)
Y = np.load(ALL_SUBJECTS_NPY_LABEL_PATH)
graph_data = generate_subject_graphs(X, Y, subject="all")

/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


#### Create a batch of DGLGraphs
*A batch of graphs can be viewed as a large graph that has many disjoint connected components*


In [40]:
def collate(graphs):
  """
  Function to create a batched graph from a collection od DGLGraph objects.
  Return:
    - batched_graph: type(DGLGraph object)
  """
  batched_graph = dgl.batch(graphs)
  return batched_graph

In [41]:
def generate_graph_batches(graphs:list, labels, batch_size=10):
  """
  Arguments:
    - graphs: type(list of DGLGraph objects),  shape(num_samples,)
    - labels: type(list of per sample labels), shape(num_samples,)

  Return:
    - batches: type(ndarray of DGLGraph objects),  shape(num_samples//batch_size,) - array of batched graphs
    - labels: type(ndarray), shape(num_samples//batch_size,)
  """

  num_graphs = len(graphs)
  num_batches=num_graphs//batch_size
  # graphs = [graph.to('/gpu:0') for graph in graphs]

  X_batches, y_batches = [], []
  for i in range(0, num_graphs, batch_size):
    if i+batch_size > num_graphs: break
    X_batches.append(collate(graphs[i:i+batch_size]))
    y_batches.append(labels[i:i+batch_size])
  
  assert len(X_batches) == num_batches
  assert len(y_batches) == num_batches

  return X_batches, y_batches

In [42]:
def get_batched_train_data(batch_size=10, num_nodes=62, subject=0):
  graph_data, graph_labels = load_graphs(os.path.join(GRAPH_PATH, f"graphs_from_de_feature_subj{subject}.bin"))
  graph_labels = graph_labels['glabels'].numpy()
  graph_labels = graph_labels.tolist()
  assert (len(graph_data) == len(graph_labels))

  X_train, X_test, y_train, y_test = train_test_split(graph_data, graph_labels, test_size=0.2, random_state=42)
  
  X_train, y_train = generate_graph_batches(X_train, y_train)
  X_test, y_test = generate_graph_batches(X_test, y_test)

  for i, batched_graph in enumerate(X_train):
    assert (batched_graph.number_of_nodes() == num_nodes * batch_size)

  return X_train, X_test, y_train, y_test

In [43]:
X_train, X_test, y_train, y_test = get_batched_train_data()

In [44]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

162 162
40 40


In [46]:
class GCN(tf.keras.Model):
  def __init__(self, 
               in_dim, 
               out_dim,
               activation=tf.nn.relu, 
               dropout=0.5, 
               pooling=True, 
               hidden_dim=32):
    
    super(GCN, self).__init__()
    self.in_dim = in_dim
    self.glayers = []
    self.glayers.append(GraphConv(in_dim, hidden_dim, activation=activation))
    self.glayers.append(GraphConv(hidden_dim, hidden_dim, activation=activation))
    self.dropout = tf.keras.layers.Dropout(dropout)
    self.logits = tf.keras.layers.Dense(out_dim)

  def call(self, g):
    h = g.ndata['x']
    for i, layer in enumerate(self.glayers):
      if i != 0:
        h = self.dropout(h)
      h = layer(g, h)

    g.ndata['h'] = h
    hg = dgl.mean_nodes(g, 'h')
    return self.logits(hg)

In [61]:
class Trainer(object):
  def __init__(self, loss_fcn, optimizer, num_epochs, model):
    self.loss = loss_fcn
    self.optimizer = optimizer
    self.num_epochs = num_epochs
    self.model = model
    self.best_loss = None
  
  def eval(self, epoch, data):
    X_test, y_test = data
    eval_loss_avg = tf.keras.metrics.Mean()
    eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for X, y in zip(X_test, y_test):
      logits = self.model(X)
      loss_val = self.loss(y, logits)
      eval_loss_avg.update_state(loss_val)
      eval_accuracy.update_state(y, logits)
    if self.best_loss is None or (eval_loss_avg.result() < self.best_loss):
      self.model.save_weights(CHECKPOINTS_PATH)
    print("Epoch {:03d}: Eval Loss: {:.3f}, Eval Accuracy: {:.3%}".format(epoch, eval_loss_avg.result(), eval_accuracy.result()))

  def run(self, data):
    train_loss = []
    train_acc = []

    X_train, y_train, X_test, y_test = data
    for epoch in range(self.num_epochs):
      epoch_loss_avg = tf.keras.metrics.Mean()
      epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
      
      for X, y in zip(X_train, y_train):
        with tf.GradientTape() as tape:
          logits = self.model(X)
          loss_val = self.loss(y, logits)
          grads = tape.gradient(loss_val, model.trainable_weights)
          self.optimizer.apply_gradients(zip(grads, model.trainable_weights))

        epoch_loss_avg.update_state(loss_val)
        epoch_accuracy.update_state(y, logits)

      train_loss.append(epoch_loss_avg.result())
      train_acc.append(epoch_accuracy.result())
      if epoch % 20 == 0:
        print("Epoch {:03d}: Train Loss: {:.3f}, Train Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))
        self.eval(epoch, (X_test, y_test))


In [62]:
# subject-dependent acc ~67%
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
num_features = 25
model = GCN(in_dim=num_features, out_dim=num_classes)

trainer = Trainer(loss_fcn, optimizer, num_epochs=200, model=model)
trainer.run((X_train, y_train, X_test, y_test))

Epoch 000: Train Loss: 2.437, Train Accuracy: 34.383%
Epoch 000: Eval Loss: 1.176, Eval Accuracy: 31.500%
Epoch 020: Train Loss: 1.007, Train Accuracy: 48.580%
Epoch 020: Eval Loss: 0.939, Eval Accuracy: 57.000%
Epoch 040: Train Loss: 0.927, Train Accuracy: 54.012%
Epoch 040: Eval Loss: 0.888, Eval Accuracy: 60.500%
Epoch 060: Train Loss: 0.860, Train Accuracy: 57.963%
Epoch 060: Eval Loss: 0.811, Eval Accuracy: 61.750%
Epoch 080: Train Loss: 0.828, Train Accuracy: 58.457%
Epoch 080: Eval Loss: 0.798, Eval Accuracy: 61.250%
Epoch 100: Train Loss: 0.796, Train Accuracy: 61.728%
Epoch 100: Eval Loss: 0.740, Eval Accuracy: 64.750%
Epoch 120: Train Loss: 0.787, Train Accuracy: 61.111%
Epoch 120: Eval Loss: 0.773, Eval Accuracy: 62.000%
Epoch 140: Train Loss: 0.771, Train Accuracy: 61.481%
Epoch 140: Eval Loss: 0.772, Eval Accuracy: 63.750%
Epoch 160: Train Loss: 0.763, Train Accuracy: 62.037%
Epoch 160: Eval Loss: 0.730, Eval Accuracy: 66.750%
Epoch 180: Train Loss: 0.766, Train Accuracy: 